In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

print("Packages imported.")

Packages imported.


In [2]:
mnist = input_data.read_data_sets("data/", one_hot = True)
train_data, train_label, test_data, test_label = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
ntrain, ntest, dim, nclasses = train_data.shape[0], test_data.shape[0], train_data.shape[1], train_label.shape[1]
print("MNIST loaded.")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded.


In [3]:
input_dim = 28
hidden_dim = 128
output_dim = nclasses
steps = 28
weights = {
    "hidden": tf.Variable(tf.random_normal([input_dim, hidden_dim])),
    "output": tf.Variable(tf.random_normal([hidden_dim, output_dim]))
}

biases = {
    "hidden": tf.Variable(tf.random_normal([hidden_dim])),
    "output": tf.Variable(tf.random_normal([output_dim]))
}

In [20]:
def RNN(input_data, weights, biases, steps, name):
    input_data = tf.transpose(input_data, [1, 0, 2])
    input_data = tf.reshape(input_data, [-1, input_dim])
    hidden = tf.matmul(input_data, weights["hidden"]) + biases["hidden"]
    hidden_split = tf.split(hidden, steps, 0)
    
    with tf.variable_scope(name) as scope:
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_dim, forget_bias=1.0)
        lstm_output, lstm_temp = tf.contrib.rnn.RNNCell(lstm_cell, hidden_split, tf.float32)
        
        output = tf.matmul(lstm_output[-1], weights["output"]) + biases["output"]
    
    return {
        "input_data": input_data, "hidden": hidden, "hidden_split": hidden_split,
        "lstm_output": lstm_output, "lstm_temp": lstm_temp, "output": output
    }
print("Network ready.")

Network ready.


In [21]:
learning_rate = 0.001
data = tf.placeholder(tf.float32, [None, steps, input_dim])
label = tf.placeholder(tf.float32, [None, output_dim])
myrnn = RNN(data, weights, biases, steps, "basic")
pred = myrnn["output"]
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, label))
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), tf.float32))

init = tf.global_variables_initializer()

print("Network ready.")

TypeError: 'RNNCell' object is not iterable